## Evolution of Embedding Layer

### TODO:
- [ ] Add callback 
- [ ] Store training steps
- [ ] Extract embedding weights
- [ ] Calculate SVD
- [ ] Project it on some human percivable space
- [ ] Present the evolution (rotation) of vectors

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

import numpy as np

In [2]:
num_tokens = 10000
(x_train_raw, y_train ), (x_test_raw, y_test) = keras.datasets.imdb.load_data(num_words=num_tokens)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
words_dict = keras.datasets.imdb.get_word_index()
len(words_dict)

1654784/1641221 [==============================] - 0s 0us/step


88584

In [4]:
x_train_raw.shape, x_test_raw.shape

((25000,), (25000,))

In [5]:
# Entries (reviews) have different length 
len(x_train_raw[0]), len(x_train_raw[1])

(218, 189)

In [6]:
max_len = 250
x_train = keras.preprocessing.sequence.pad_sequences(x_train_raw, maxlen=max_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test_raw, maxlen=max_len)


In [7]:

x_train.shape, x_test.shape

((25000, 250), (25000, 250))

In [8]:
# How to convert numpy array to a dataset tensor

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices ((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_ds = train_ds.shuffle(train_ds.cardinality())
test_ds = test_ds.shuffle(test_ds.cardinality())
batch_size = 32
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [10]:
train_ds.cardinality(), test_ds.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=782>,
 <tf.Tensor: shape=(), dtype=int64, numpy=782>)

In [11]:
def create_mode(max_len=max_len, 
                num_tokens=num_tokens,
                embedding_size=16):
  
  """ Create a sequencial vanilla deeplearning model.
  """
  inputs = layers.Input(shape=(None,), dtype=tf.int64)
  x = layers.Embedding(num_tokens, embedding_size)(inputs)
  x = layers.GlobalAvgPool1D()(x)
  x = layers.Dense(units=32, activation='relu')(x)
  x = layers.Dropout(0.2)(x)
  outputs = layers.Dense(units=1, activation='relu')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)

  return model

In [12]:
model = create_mode()

In [13]:
model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 32)                544       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                             

In [15]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='path/to/my/model_{epoch}',
        save_freq='epoch')
]

In [16]:
epochs=10
model.fit(train_ds,
          validation_data=test_ds,
          epochs=epochs,
          callbacks=callbacks)

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.6034 - accuracy: 0.6980 - val_loss: 0.3835 - val_accuracy: 0.8590
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3499 - accuracy: 0.8797 - val_loss: 0.3414 - val_accuracy: 0.8807
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2850 - accuracy: 0.9074 - val_loss: 0.3607 - val_accuracy: 0.8822
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3229 - accuracy: 0.8974 - val_loss: 0.3893 - val_accuracy: 0.8814
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2920 - accuracy: 0.9064 - val_loss: 0.4189 - val_accuracy: 0.8549
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2396 - accuracy: 0.9268 - val_loss: 0.5016 - val_accuracy: 0.8790
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2166 - accuracy: 0.9371 - val_loss: 0.5464 - val_accuracy: 0.8763
Epoch 

In [17]:
embed_l = model.get_layer('embedding_5')

ValueError: ignored

In [ ]:
embd_mtx = embed_l.embeddings

In [ ]:
embd_mtx.shape

In [ ]:
mtx_embd = tf.matmul(embd_mtx, embd_mtx, transpose_a=True)

In [ ]:
from numpy import linalg as lng

In [ ]:
svd = lng.svd(embd_mtx)

In [ ]:
len(svd)

In [ ]:
svd[0].shapem 